In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/2nd-leader-model-from-automl/StackedEnsemble_BestOfFamily_1_AutoML_1_20241013_192134
/kaggle/input/stacked-ensemble-leader-model-from-h2o-automl/StackedEnsemble_AllModels_1_AutoML_1_20241013_192134
/kaggle/input/playground-series-s4e10/sample_submission.csv
/kaggle/input/playground-series-s4e10/train.csv
/kaggle/input/playground-series-s4e10/test.csv


In [2]:
test = pd.read_csv('/kaggle/input/playground-series-s4e10/test.csv')
idx = test['id']
test.drop(columns='id', inplace=True)

In [3]:
import h2o
from h2o.automl import H2OAutoML

h2o.init()
h2o_df = h2o.H2OFrame(test)

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.24" 2024-07-16; OpenJDK Runtime Environment (build 11.0.24+8-post-Ubuntu-1ubuntu320.04); OpenJDK 64-Bit Server VM (build 11.0.24+8-post-Ubuntu-1ubuntu320.04, mixed mode, sharing)
  Starting server from /opt/conda/lib/python3.10/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpj5wj4e02
  JVM stdout: /tmp/tmpj5wj4e02/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmpj5wj4e02/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,03 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.5
H2O_cluster_version_age:,1 month and 14 days
H2O_cluster_name:,H2O_from_python_unknownUser_782mji
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,7.500 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"locked, healthy"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [4]:
leader_model = h2o.load_model('/kaggle/input/2nd-leader-model-from-automl/StackedEnsemble_BestOfFamily_1_AutoML_1_20241013_192134')

In [5]:
preds = leader_model.predict(h2o_df)[:,2]

stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%


In [6]:
preds = preds.as_data_frame()

/opt/conda/lib/python3.10/site-packages/h2o/frame.py:1981: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install polars and pyarrow and use it as pandas_df = h2o_df.as_data_frame(use_multi_thread=True)

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"


In [7]:
preds

,p1
0,0.996272
1,0.015739
2,0.548539
3,0.007830
4,0.073981
...,...
39093,0.052943
39094,0.003409
39095,0.016417
39096,0.439397


In [8]:
submission_df = pd.DataFrame({
    'id':idx,
    'loan_status':preds['p1'].values
})

In [9]:
submission_df.to_csv('submission.csv', index=False)

In [10]:
h2o.shutdown()

H2O session _sid_815d closed.


/tmp/ipykernel_17/2900054706.py:1: H2ODeprecationWarning: Deprecated, use ``h2o.cluster().shutdown()``.
  h2o.shutdown()
